In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns',500)

In [20]:
df = pd.read_excel("/content/final_second.xlsx")

In [21]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,index,name,Total
2,NaN,NaN,NaN,Solar1,1993.159327
3,NaN,NaN,NaN,Solar2,738.321828
4,NaN,NaN,0,Agni Cool_Floor-1_A Block,166.543643


In [22]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [23]:
df = df.drop(['Unnamed: 0','Unnamed: 1'],axis = 1)

In [24]:
df.columns = [['index','name','energy KwH']]

In [25]:
df = df.iloc[2:]

In [26]:
df.head()

,index,name,energy KwH
2,NaN,Solar1,1993.159327
3,NaN,Solar2,738.321828
4,0,Agni Cool_Floor-1_A Block,166.543643
5,1,Agni Cool_Floor-1_E Block,0.179521
6,2,Agni Cool_Floor-3_E Block,8.585915


In [11]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 27.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.24.3 which is incompatible.


In [12]:
df.head()

,index,name,energy KwH
3,NaN,Solar1,1993.159327
4,NaN,Solar2,738.321828
5,0,Agni Cool_Floor-1_A Block,166.543643
6,1,Agni Cool_Floor-1_E Block,0.179521
7,2,Agni Cool_Floor-3_E Block,8.585915


In [27]:
data_part = df[['index','name','energy KwH']].iloc[2:].to_dict(orient = 'records')

In [28]:
data_part

[{('index',): 0,
  ('name',): 'Agni Cool_Floor-1_A Block',
  ('energy KwH',): 166.54364337103365},
 {('index',): 1,
  ('name',): 'Agni Cool_Floor-1_E Block',
  ('energy KwH',): 0.1795211455495146},
 {('index',): 2,
  ('name',): 'Agni Cool_Floor-3_E Block',
  ('energy KwH',): 8.585914650998719},
 {('index',): 3,
  ('name',): 'ARCI_Floor-2_E Block',
  ('energy KwH',): 153.20247269391075},
 {('index',): 4,
  ('name',): 'ARCI_Floor-7_E Block',
  ('energy KwH',): 118.7377667136929},
 {('index',): 5,
  ('name',): 'ARCI_Floor-8_E Block',
  ('energy KwH',): 87.70946879368344},
 {('index',): 6,
  ('name',): 'BioIncubator_Floor-3_E Block',
  ('energy KwH',): 25.18131611080603},
 {('index',): 7,
  ('name',): 'BioIncubator_Floor-5_C Block',
  ('energy KwH',): 38.60200969113563},
 {('index',): 8,
  ('name',): 'CaterPillar_Floor-7_E Block',
  ('energy KwH',): 145.4321612707922},
 {('index',): 9,
  ('name',): 'Cewit_Floor-5_A Block',
  ('energy KwH',): 108.95896664240526},
 {('index',): 10,
  ('name'

In [29]:
dt_val = [round(dt[('energy KwH',)]) for dt in data_part]

In [30]:
data_sol = df[['index','name','energy KwH']].iloc[:2].to_dict(orient = 'records')

In [31]:
dt_val_solar = [round(dt[('energy KwH',)]) for dt in data_sol]

In [32]:
dt_val_solar

[1993, 738]

In [33]:
"""Solve a multiple knapsack problem using a MIP solver."""
from ortools.linear_solver import pywraplp

l1 = []
def main():
    data = {}
    data["weights"] = dt_val
    data["values"] = dt_val
    assert len(data["weights"]) == len(data["values"])
    data["num_items"] = len(data["weights"])
    data["all_items"] = range(data["num_items"])

    data["bin_capacities"] = dt_val_solar
    data["num_bins"] = len(data["bin_capacities"])
    data["all_bins"] = range(data["num_bins"])

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if solver is None:
        print("SCIP solver unavailable.")
        return

    # Variables.
    # x[i, b] = 1 if item i is packed in bin b.
    x = {}
    for i in data["all_items"]:
        for b in data["all_bins"]:
            x[i, b] = solver.BoolVar(f"x_{i}_{b}")

    # Constraints.
    # Each item is assigned to at most one bin.
    for i in data["all_items"]:
        solver.Add(sum(x[i, b] for b in data["all_bins"]) <= 1)

    # The amount packed in each bin cannot exceed its capacity.
    for b in data["all_bins"]:
        solver.Add(
            sum(x[i, b] * data["weights"][i] for i in data["all_items"])
            <= data["bin_capacities"][b]
        )

    # Objective.
    # Maximize total value of packed items.
    objective = solver.Objective()
    for i in data["all_items"]:
        for b in data["all_bins"]:
            objective.SetCoefficient(x[i, b], data["values"][i])
    objective.SetMaximization()

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print(f"Total packed value: {objective.Value()}")
        total_weight = 0
        for b in data["all_bins"]:
            print(f"Bin {b}")
            bin_weight = 0
            bin_value = 0
            mat = np.zeros(26)
            for i in data["all_items"]:
                if x[i, b].solution_value() > 0:
                    print(
                        f"item {[i]}{data_part[i][('name',)]} weight: {data['weights'][i]} value: {data['values'][i]}"
                    )
                    mat[i] = 1
                    bin_weight += data["weights"][i]
                    bin_value += data["values"][i]

            print(f"Packed bin weight: {bin_weight}")
            print(f"Packed bin value: {bin_value}\n")
            l1.append(mat)
            print(mat)
            total_weight += bin_weight
        print(f"Total packed weight: {total_weight}")
    else:
        print("The problem does not have an optimal solution.")


if __name__ == "__main__":
    main()

Total packed value: 2328.0
Bin 0
item [0]Agni Cool_Floor-1_A Block weight: 167 value: 167
item [2]Agni Cool_Floor-3_E Block weight: 9 value: 9
item [3]ARCI_Floor-2_E Block weight: 153 value: 153
item [4]ARCI_Floor-7_E Block weight: 119 value: 119
item [5]ARCI_Floor-8_E Block weight: 88 value: 88
item [6]BioIncubator_Floor-3_E Block weight: 25 value: 25
item [7]BioIncubator_Floor-5_C Block weight: 39 value: 39
item [8]CaterPillar_Floor-7_E Block weight: 145 value: 145
item [9]Cewit_Floor-5_A Block weight: 109 value: 109
item [10]Cewit_Floor-6_A Block weight: 81 value: 81
item [11]DRDO_Floor-5_E Block weight: 48 value: 48
item [12]HTIC_Floor-5_C Block weight: 67 value: 67
item [13]ICCW_Ground Floor_B Block weight: 175 value: 175
item [14]IFMR_Floor-7_B Block weight: 10 value: 10
item [15]IFMR_Floor-8_E Block weight: 84 value: 84
item [16]IIT Alumni_Floor-10_C Block weight: 122 value: 122
item [17]NMS_Floor-4_E Block weight: 28 value: 28
item [18]NMS_Floor-6_E Block weight: 83 value: 83
i

In [ ]:
pd.DataFrame(l1).to_csv('list_2nd.csv')